In [ ]:
import pyomo.environ as pe
import pyomo.opt as po

In [ ]:
num_days = 5
levels = {'lo', 'md', 'hi'}
cost = 5
revenue = {1: 8, 2: 7, 3: 6, 4: 5, 5: 4}
probability = {'lo': 0.25, 'md': 0.50, 'hi': 0.25}
demand = {'lo':  8, 'md': 10, 'hi': 12}

In [ ]:
# num_days = 2
# levels = {'lo', 'hi'}
# cost = 5
# revenue = {1: 7, 2: 5}
# probability = {'lo': 0.25, 'hi': 0.75}
# demand = {'lo': 5, 'hi': 10}

In [ ]:
root = pe.ConcreteModel()
root.levels = pe.Set(initialize=levels)
root.cost = pe.Param(initialize=cost)
root.day = 0
root.probability = 1
root.x = pe.Var(bounds=(0, demand['hi'] * 5))

def blk_child(block, level):
    parent = block.parent_block()
    block.levels = pe.Set(initialize=levels)
    block.day = parent.day + 1
    block.probability = parent.probability * probability[level]
    block.demand = demand[level]
    block.revenue = revenue[block.day]
    block.y = pe.Var(bounds=(0, block.demand))
    block.x = pe.Var()
    if block.day < num_days:
        block.child = pe.Block(block.levels, rule=blk_child)
    block.con_y_le_x = pe.Constraint(expr=block.y <= parent.x)
    block.con_x = pe.Constraint(expr=block.x == parent.x - block.y)

root.child = pe.Block(root.levels, rule=blk_child)

def obj(model):
    expr = 0
    for block_obj in model.component_objects(pe.Block, active=True):
        for _, block in block_obj.items():
            expr += block.probability * block.revenue * block.y
    expr -= model.cost * model.x
    return expr

root.obj = pe.Objective(sense=pe.maximize, rule=obj)

In [ ]:
solver = po.SolverFactory('gurobi')
solver.options['mipgap'] = 0.0
result = solver.solve(root)
print(pe.value(root.obj))
print(pe.value(root.x))

In [ ]:
from collections import Counter
from itertools import product
from operator import mul
import numpy as np
from functools import reduce

def dec_along_path(root, path):
    sol = []
    block = root
    for level in path:
        block = block.child[level]
        sol.append(int(pe.value(block.y)))
    return sol

profit = 0
for path in product(levels, repeat=num_days):
    sol = np.array(dec_along_path(root, path))
    p = reduce(mul, [probability[level] ** ct for level, ct in Counter(path).items()])
    profit += p * np.array([*revenue.values()]) @ sol
profit -= pe.value(root.x) * cost
print(profit)

In [ ]:
def expected_profit(x):
    profit = 0
    for path in product(levels, repeat=num_days):
        x_copy = x
        path_revenue = 0
        path_probability = 1.0
        for day, level in enumerate(path, start=1):
            sold = min(x_copy, demand[level])
            path_revenue += sold * revenue[day]
            path_probability *= probability[level]
            x_copy -= sold
        profit += path_probability * path_revenue
    profit -= cost * x
    return profit

In [ ]:
import matplotlib.pyplot as plt

# x = 0
# y = expected_profit(x)
# while y >= 0:
#     plt.plot(x, y, 'ro')
#     x += 1
#     y = expected_profit(x)
# plt.show()

for x in range(30, 40):
    y = expected_profit(x)
    plt.plot(x, y, 'ro')
plt.show()